# **DSML Assignment 3**
* **Name**: Narendra Jadhav
* **Roll No.**: 324022
* **GR No.**: 22010817
* **Div**: D
* **Batch**: D1

* **Problem Statement**: 
Write a program to do: A dataset collected in a cosmetics shop showing details of customers and whether they responded to a special offer to buy a new lip-stick is shown in table below.
Use this dataset to build a decision tree, with Buys as the target variable, to help in buying lipsticks in the future. Find the root node of decision tree. According to the decision tree you have made from previous training data set, what is the decision for the test data: [Age < 21, Income = Low, Gender = Female, Marital Status = Married]?

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_excel("Decision.xlsx")
df

,Id,Age,Income,Gender,Ms,Buys
0,1.0,<21,High,Male,Single,No
1,2.0,<21,High,Male,Married,No
2,3.0,21-35,High,Male,Single,Yes
3,4.0,>35,Medium,Male,Single,Yes
4,5.0,>35,Low,Female,Single,Yes
5,6.0,>35,Low,Female,Married,No
6,7.0,21-35,Low,Female,Married,Yes
7,8.0,<21,Medium,Male,Single,No
8,9.0,<21,Low,Female,Married,Yes
9,10.0,>35,Medium,Female,Single,Yes


In [ ]:
for x in range(len(df["Age"])):
    df["Age"].replace("<21", 0, inplace=True)
    df["Age"].replace("21-35", 1, inplace=True)
    df["Age"].replace(">35", 2, inplace=True)
    
    df["Income"].replace("High", 2, inplace=True)
    df["Income"].replace("Medium", 1, inplace=True)
    df["Income"].replace("Low", 0, inplace=True)
    
    df["Gender"].replace("Male", 0, inplace=True)
    df["Gender"].replace("Female", 1, inplace=True)
    
    df["Buys"].replace("Yes", 1, inplace=True)
    df["Buys"].replace("No", -1, inplace=True)
    
    df["Ms"].replace("Single", 1, inplace=True)
    df["Ms"].replace("Married", 2, inplace=True)

In [ ]:
df.columns

Index(['Id', 'Age', 'Income', 'Gender', 'Ms', 'Buys'], dtype='object')

# **Decision Tree**

In [ ]:
def entropy(c):
    ent = 0.0
    c, counts = np.unique(c, return_counts=True)
    for i in range(len(c)):
        prob = (counts[i]) / float(len(counts))
        ent += prob * np.log2(prob)
    return -ent
 

def divide_data(x_data, f_key, f_val):
    
    left = pd.DataFrame([],columns=x_data.columns )
    right = pd.DataFrame([],columns=x_data.columns)
    for i in range(x_data[f_key].shape[0]):
        val = x_data[f_key][i] #try withx-dats[afkey][i]
        if(val < f_val):
            left = left.append(x_data.loc[i]) #iloc
        else:
            right = right.append(x_data.loc[i])#iloc
    
    return left, right

def information_gain(x_data, f_val, f_key):
    left, right = divide_data(x_data, f_key, f_val)
    l = float (left.shape[0]) / x_data.shape[0]
    r = float (right.shape[0]) / x_data.shape[0]
    hs = entropy(x_data.Buys)
    ans = hs - (l * entropy(left.Buys) + r * entropy(right.Buys))
    return ans


In [ ]:
class CustomDT:
    def __init__(self, depth = 0, max_depth = 5):
        self._max_depth = max_depth
        self._depth = depth
        self.right = None
        self.left = None
        self.f_key = None
        self.f_val = None
        self.target = None
        
    def fit(self, X):
        info_gain = []
        features = ['Age', 'Income', 'Gender', 'Ms']
        for i in features:
            info_gain.append(information_gain(X, X[i].mean(), i))
      
        self.f_key = features[np.argmax(info_gain)]
        self.f_val = X[self.f_key].mean()
        left, right = divide_data(X, self.f_key, self.f_val)
        left = left.reset_index(drop=True)
        right = right.reset_index(drop=True)
        if (left.shape[0] == 0 or right.shape[0] == 0):
            if(X.Buys.mean() >= 0.5):
                self.target = "Buys"
            else:
                self.target = "Doesn't Buy"
            return
        
        if(self._depth >= self._max_depth):
            if(X.Buys.mean() >= 0.5):
                self.target = "Buys"
            else:
                self.target = "Doesn't Buy"
            return

        print("Nodes: ", self.f_key)
        self.left = CustomDT(depth=self._depth + 1)
        self.left.fit(left)
        self.right = CustomDT(depth=self._depth + 1)
        self.right.fit(right)
        

        if X.Buys.mean() >= 0.5:
            self.target = "Buys"
        else:
            self.target = "Doesn't Buy"
        return
        
    def predict(self, test):
        result = []
        if test[self.f_key] > self.f_val:
            # go to right
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [ ]:
dt = CustomDT()

# **Prepare Data**

In [ ]:
split = int(0.7 * df.shape[0])

In [ ]:
X_train = df[:split]
X_test = df[split:]
X_test = X_test.reset_index(drop=True)

# **Test / Predict**

In [ ]:
dt.fit(X_train)

Nodes:  Age
Nodes:  Gender
Nodes:  Ms
Nodes:  Age


In [ ]:
result = []
for i in range(X_test.shape[0]):
    result.append(dt.predict(X_test.iloc[i]))

In [ ]:
X_test.iloc[0]

Id        10.0
Age        2.0
Income     1.0
Gender     1.0
Ms         1.0
Buys       1.0
Name: 0, dtype: float64

In [ ]:
result ## Predicted output
dict = {
    "Buys": 1,
    "Doesn't Buy" : -1
}

result_converted_to_labels = []

for x in result:
    result_converted_to_labels.append(dict[x])
result_converted_to_labels

[1, 1, 1, 1, -1]

In [ ]:
df[split:] ## Actual Output

,Id,Age,Income,Gender,Ms,Buys
9,10.0,2,1,1,1,1
10,11.0,0,1,1,2,1
11,12.0,1,1,0,2,1
12,13.0,1,2,1,1,1
13,14.0,2,1,0,2,-1


In [ ]:
df

,Id,Age,Income,Gender,Ms,Buys
0,1.0,0,2,0,1,-1
1,2.0,0,2,0,2,-1
2,3.0,1,2,0,1,1
3,4.0,2,1,0,1,1
4,5.0,2,0,1,1,1
5,6.0,2,0,1,2,-1
6,7.0,1,0,1,2,1
7,8.0,0,1,0,1,-1
8,9.0,0,0,1,2,1
9,10.0,2,1,1,1,1


# **Decision for the test data: [Age < 21, Income = Low, Gender = Female, Marital Status = Married]**

In [ ]:
test_data = pd.DataFrame([])

for x in range(len(df)):
    if df["Age"].iloc[x] == 0 and df["Income"].iloc[x] == 0 and df["Gender"].iloc[x] == 1 and df["Ms"].iloc[x] == 2:
        test_data = test_data.append(df.iloc[x])

In [ ]:
test_data

,Id,Age,Income,Gender,Ms,Buys
8,9.0,0.0,0.0,1.0,2.0,1.0


In [ ]:
res = []
test_data = test_data.reset_index(drop=True)
for x in range(test_data.shape[0]):
    res.append(dt.predict(test_data.iloc[x]))
res

['Buys']

***Hence, according to the decision tree, the decision for a married woman with an age less than 21 and low income, will 'Buy' the lipstick.***